In [ ]:
WDPA_Ia = snakemake.input.WDPA1a
WDPA_Ib = snakemake.input.WDPA1b
WDPA_II = snakemake.input.WDPA2
WDPA_III = snakemake.input.WDPA3
WDPA_IV = snakemake.input.WDPA4

# heightshp = snakemake.input.elevation
# degreeshp = snakemake.input.slope

heightshp = ""
degreeshp = ""

weatherdata = snakemake.input.weatherdata

desired_regions = snakemake.params.aggregated_regions

# Geodata files to use for selecting country onshore and offshore area:
geodata_files = {
    "onshore": snakemake.input.euroshape,
    "offshore_bottom": snakemake.input.eurooffshoreshape,
}

cfdata = snakemake.input.cfdata

CORINE = snakemake.input.corine

grid_areas_solar = snakemake.output.grid_areassolar
grid_areas_windon = snakemake.output.grid_areaswindonshore
grid_areas_windoff = snakemake.output.grid_areaswindoffshore

In [ ]:
codes = (
    2,
    4,
    5,
    6,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
)

In [ ]:
wind_onshore_codes_no_buffer = (
    3,
    4,
    5,
    7,
    8,
    9,
    10,
    11,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
)

wind_onshore_codes_buffer = {1: 2000, 2: 1000, 6: 5000}

In [ ]:
panel = "CSi"
orientation = "latitude_optimal"

file_name = geodata_files["onshore"]

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

import io
import os
import pathlib

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xarray as xr
from atlite.gis import ExclusionContainer, shape_availability
from shapely.geometry import Polygon

In [ ]:
atlite.__version__  # should be 0.2.4

In [ ]:
xr.__version__  # should be 0.18.2

In [ ]:
europe=(gpd.read_file(geodata_files["onshore"])
        .set_index(["index"]))

In [ ]:
boundaries = []
for geodata_file_name, geodata_file_path in geodata_files.items():
    print(geodata_file_path)
    boundaries.append(
        gpd.read_file(geodata_file_path)
        .set_index(["index"])
    )

boundaries = pd.concat(boundaries).bounds

In [ ]:
boundaries = boundaries.groupby(lambda x: "bountry").agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
)

boundaries

In [ ]:
cutout = atlite.Cutout(path=weatherdata)
cutout.prepared_features
cutout.prepare()

In [ ]:
cutout = atlite.Cutout(
    path="../3_intermediate_data/intermediatecutout.nc",
    data=cutout.data.sel(
        x=slice(
            boundaries.loc["bountry", "minx"],
            boundaries.loc["bountry", "maxx"],
        ),
        y=slice(
            boundaries.loc["bountry", "miny"],
            boundaries.loc["bountry", "maxy"],
        ),
    ),
)

cutout.prepare()

In [ ]:
gridcellnamingfunction = (
    lambda x: "x"
    + (x.x * 100).astype("int").astype("str")
    + "y"
    + (x.y * 100).astype("int").astype("str")
)

if snakemake.wildcards.spatial == "grid":
    (
        cutout.grid.assign(gridcell=gridcellnamingfunction)
        .loc[:, ["gridcell"]]
        .to_csv(snakemake.output["indreg"], header=False, index=False)
    )
else:
    with open(snakemake.output["indreg"], "w"):
        pass

In [ ]:
def cutoff_raster(cf, cutoff, output):
    # if technology == 'pv':
    #     cf = cutout.pv(panel=panel, orientation=orientation, capacity_factor=True)
    #     smallestincluded = snakemake.params.cutoffs["solar"]

    # if technology == 'onwind':
    #     cf = cutout.wind(turbine=onshore_turbine, capacity_factor=True)
    #     smallestincluded = snakemake.params.cutoffs["onwind"]

    # if technology == 'offwind':
    #     cf = cutout.wind(turbine=offshore_bottom_turbine, capacity_factor=True)
    #     smallestincluded = snakemake.params.cutoffs["offwind"]

    excluded = xr.where(cf >= cutoff, 0, 1)

    # excluded = cf.where(cf.values>=cutoff, other=1)
    # excluded = excluded.where(cf.values<cutoff, other=0)
    cf_exclusion = excluded.rio.write_crs(europe.crs)

    cf_exclusion.rio.to_raster(output)

# Solar areas

In [ ]:
excluder_solar = ExclusionContainer()

In [ ]:
excluder_solar.add_raster(CORINE, codes=codes)
excluder_solar.add_raster(WDPA_Ia)
excluder_solar.add_raster(WDPA_Ib)
excluder_solar.add_raster(WDPA_II)
excluder_solar.add_raster(WDPA_III)
excluder_solar.add_raster(WDPA_IV)

In [ ]:
if degreeshp != "":
    slope = gpd.read_file(degreeshp).to_crs(excluder_solar.crs)

    slope = slope[slope["gridcode"] == 1]

    # use invert=True to exclude the areas that are to steep instead of
    # excluding everything else
    excluder_solar.add_geometry(slope.geometry, invert=True)

In [ ]:
if heightshp != "":
    height = gpd.read_file(heightshp).to_crs(excluder_solar.crs)

    height = height[height["gridcode"] == 1]

    excluder_solar.add_geometry(height.geometry, invert=True)

In [ ]:
if snakemake.wildcards.spatial != "grid" and snakemake.params.cutoffs["solar"] != 0:
    cf = xr.open_dataarray(cfdata)

    low_cf = snakemake.output.cf_exclusion_solar
    cutoff_raster(
        cf.loc["Solar", :, :, :].mean(dim="time"),
        snakemake.params.cutoffs["solar"],
        low_cf,
    )
    excluder_solar.add_raster(low_cf)

In [ ]:
excluder_solar

In [ ]:
availability_matrix_solar = cutout.availabilitymatrix(
    europe, excluder_solar, nprocesses=snakemake.threads
)

In [ ]:
availability_matrix_solar

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_solar = (
    availability_matrix_solar.stack(spatial=["x", "y"]) * area
)

# comparing the cap matrix before and after, the below is not required
# as all it does is reorder the array and breaks the latest xarray 
# (v2024.03.0).

# capacity_matrix_solar = capacity_matrix_solar.reindex(
#    spatial=area.indexes.get("spatial")
# )


highRESareasSolar = (
    capacity_matrix_solar.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .replace()
    .assign(gridcell=gridcellnamingfunction)
    .rename(columns={0: "area"})
    .query("area != 0")
    .merge(europe, left_on="index", right_on="index")
    .rename(columns={"CNTR_CODE": "zone"})
    .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
)

highRESareasSolar.round(1).to_csv(grid_areas_solar, index=False)

In [ ]:
# if snakemake.wildcards.spatial == "region":
#     highRESareasSolar = (
#         capacity_matrix_solar.sum("spatial")
#         .to_pandas()
#         .reset_index()
#         .assign(
#             new_idx=lambda x: "Solar." + x["index"] + "." + x["index"],
#         )
#         .set_index("new_idx")
#         .drop(columns=["index"])
#         .loc[:, [0]]
#         .rename(columns={0: "area"})
#         .query("area != 0")
#     )

In [ ]:
# Onshore wind areas

In [ ]:
excluder_wind_onshore = ExclusionContainer()

In [ ]:
if degreeshp != "":
    slope = gpd.read_file(degreeshp).to_crs(excluder_wind_onshore.crs)

    slope = slope[slope["gridcode"] == 1]

    excluder_wind_onshore.add_geometry(slope.geometry, invert=True)
    # use invert=True to exclude the areas that are to steep instead of
    # excluding everything else

In [ ]:
if heightshp != "":
    height = gpd.read_file(heightshp).to_crs(excluder_wind_onshore.crs)

    height = height[height["gridcode"] == 1]

    excluder_wind_onshore.add_geometry(height.geometry, invert=True)

In [ ]:
if snakemake.wildcards.spatial != "grid" and snakemake.params.cutoffs["onwind"] != 0:
    cf = xr.open_dataarray(cfdata)

    low_cf = snakemake.output.cf_exclusion_windon
    cutoff_raster(
        cf.loc["Windonshore", :, :, :].mean(dim="time"),
        snakemake.params.cutoffs["onwind"],
        low_cf,
    )
    excluder_wind_onshore.add_raster(low_cf)

In [ ]:
availability_matrix_wind_onshore = cutout.availabilitymatrix(
    europe, excluder_wind_onshore, nprocesses=snakemake.threads
)
availability_matrix_wind_onshore

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_onshore = (
    availability_matrix_wind_onshore.stack(spatial=["x", "y"]) * area
)

# comparing the cap matrix before and after, the below is not required
# as all it does is reorder the array and breaks the latest xarray 
# (v2024.03.0).

# capacity_matrix_wind_onshore = capacity_matrix_wind_onshore.reindex(
#    spatial=area.indexes.get("spatial")
# )

highRESareasWindOnshore = (
    capacity_matrix_wind_onshore.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .replace()
    .assign(gridcell=gridcellnamingfunction)
    .rename(columns={0: "area"})
    .query("area != 0")
    .merge(europe, left_on="index", right_on="index")
    .rename(columns={"CNTR_CODE": "zone"})
    .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
)

highRESareasWindOnshore.round(1).to_csv(grid_areas_windon, index=False)

In [ ]:
# if snakemake.wildcards.spatial == "region":
#     highRESareasWindOnshore = (
#         capacity_matrix_wind_onshore.sum("spatial")
#         .to_pandas()
#         .reset_index()
#         .assign(
#             new_idx=lambda x: "Windonshore." + x["index"] + "." + x["index"],
#         )
#         .set_index("new_idx")
#         .drop(columns=["index"])
#         .loc[:, [0]]
#         .rename(columns={0: "area"})
#         .query("area != 0")
#     )

In [ ]:
# Offshore wind areas

In [ ]:
europe_offshore_bottom = (
    gpd.read_file(geodata_files["offshore_bottom"])
)
europe_offshore_bottom

In [ ]:
excluder_wind_offshore_bottom = ExclusionContainer()

In [ ]:
excluder_wind_offshore_bottom.add_raster(WDPA_Ia)
excluder_wind_offshore_bottom.add_raster(WDPA_Ib)
excluder_wind_offshore_bottom.add_raster(WDPA_II)
excluder_wind_offshore_bottom.add_raster(WDPA_III)
excluder_wind_offshore_bottom.add_raster(WDPA_IV)

In [ ]:
if snakemake.wildcards.spatial != "grid" and snakemake.params.cutoffs["offwind"] != 0:
    cf = xr.open_dataarray(cfdata)

    low_cf = snakemake.output.cf_exclusion_windoff
    cutoff_raster(
        cf.loc["Windoffshore", :, :, :].mean(dim="time"),
        snakemake.params.cutoffs["offwind"],
        low_cf,
    )
    excluder_wind_offshore_bottom.add_raster(low_cf)

In [ ]:
grid = cutout.grid.set_index(["x", "y"]).to_crs(3035)

europe_windoff = gpd.overlay(
    europe_offshore_bottom.reset_index().to_crs(3035), grid, how="intersection"
)
europe_windoff["centroid"] = europe_windoff["geometry"].centroid


zone = []
for i, p in europe_windoff.iterrows():
    ztemp = europe.loc[europe["CNTR_CODE"] == p["index"], :].to_crs(3035)
    dists = ztemp.distance(p["centroid"]).sort_values()

    if len(dists) == 0:
        break

    zone.append(dists.index[0])

europe_windoff["zone"] = zone

# europe_windoff.loc[:,["index","zone","geometry"]].to_file("europe_windoff.geojson")

In [ ]:
europe_offshore_bottom = (
    europe_windoff.loc[:, ["index", "zone", "geometry"]]
    .dissolve(by="zone")
    .rename(columns={"index": "zone"})
    .rename_axis(index={"zone": "index"})
)

In [ ]:
availability_matrix_wind_offshore_bottom = cutout.availabilitymatrix(
    europe_offshore_bottom, excluder_wind_offshore_bottom, nprocesses=snakemake.threads
)
availability_matrix_wind_offshore_bottom

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_offshore_bottom = (
    availability_matrix_wind_offshore_bottom.stack(spatial=["x", "y"]) * area
)

# comparing the cap matrix before and after, the below is not required
# as all it does is reorder the array and breaks the latest xarray 
# (v2024.03.0).

# capacity_matrix_wind_offshore_bottom = capacity_matrix_wind_offshore_bottom.reindex(
#    spatial=area.indexes.get("spatial")
# )

highRESareasWindOffshoreBottom = (
    capacity_matrix_wind_offshore_bottom.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .replace()
    .assign(gridcell=gridcellnamingfunction)
    .rename(columns={0: "area"})
    .query("area != 0")
    .merge(europe, left_on="index", right_on="index")
    .rename(columns={"CNTR_CODE": "zone"})
    .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
)


highRESareasWindOffshoreBottom.round(1).to_csv(grid_areas_windoff, index=False)

In [ ]:
# if snakemake.wildcards.spatial == "region":
#     highRESareasWindOffshoreBottom = (
#         capacity_matrix_wind_offshore_bottom.sum("spatial")
#         .to_pandas()
#         .reset_index()
#         .assign(
#             new_idx=lambda x: "Windoffshore." + x["index"] + "." + x["index"],
#         )
#         .set_index("new_idx")
#         .drop(columns=["index"])
#         .loc[:, [0]]
#         .rename(columns={0: "area"})
#         .query("area != 0")
#     )